In [ ]:
import pandas as pd
import numpy as np
import time
import datetime
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
df_aqi = pd.read_csv('/Users/nikmag/Desktop/USC/Research/SSI/Old data/los_angeles_meo_15_months.csv',dtype={'date_observed':str})

In [ ]:
df_aqi.drop(columns=['uid','timezone','summary','icon','precip_intensity','precip_probability','uv_index','ozone','location','dew_point','apparent_temperature'],axis=1,inplace=True)

In [ ]:
df_aqi['date_observed'] = df_aqi['date_observed'].apply(lambda x: x[:-3])

In [ ]:
df_aqi['timestamp'] = df_aqi['date_observed'].apply(pd.Timestamp)

In [ ]:
df_aqi.index = df_aqi['timestamp']

In [ ]:
del df_aqi['timestamp']

In [ ]:
df_aqi.drop(columns=['date_observed'],inplace=True)

In [ ]:
df_aqi.sort_index(inplace=True)

In [ ]:
df_aqi = df_aqi.loc[:pd.Timestamp('2018-02-28 23:00:00'),:]

In [ ]:
sensors = ['Antelope Vly',
 'Central LA CO',
'E San Fernando Vly',
'E San Gabriel V-2',
'NW Coastal LA',
'SW Coastal LA',
'SW San Bernardino',
'San Gabriel Mts',
'Santa Clarita Vly',
'South Coastal LA',
'Southeast LA CO',
'W San Fernando Vly',
'W San Gabriel Vly']

In [ ]:
df_aqi.reset_index(inplace=True)

In [ ]:
df_aqi.drop_duplicates(inplace=True)

In [ ]:
df_aqi = df_aqi[df_aqi.reporting_area.isin(sensors)]

In [ ]:
df1 = pd.DataFrame('aaa',columns=['reporting_area'],index=[i for i in range(0,132288)])

In [ ]:
df1['timestamp'] = pd.Timestamp('2000-01-01 00:00:00')

In [ ]:
j=0
for i in range(0,132288):
    start = time.time()
    df1.loc[i,'reporting_area'] = common_sensors[j]
    j = j + 1
    if j == 13:
        j = 0
    if i%20000 == 0 and i!=0:
        end = time.time()
        print('iteration done in {} seconds'.format(end-start))

In [ ]:
times = pd.DataFrame(pd.date_range(start=pd.Timestamp('2017-01-01 00:00:00'),end=pd.Timestamp('2018-02-28 23:00:00'),freq='1H'),columns=['timestamp'])

In [ ]:
k = 0
for i in times['timestamp']:
    for j in range(0,13):
        df1.loc[k,'timestamp'] = i
        k = k + 1

In [ ]:
df2 = pd.merge(df1,df_aqi,how='left',on=['reporting_area','timestamp'])

In [ ]:
df2.isnull().sum()

In [ ]:
df2.fillna(-1.0,inplace=True)

In [ ]:
features = df2.columns[2:].tolist()

In [ ]:
features

In [ ]:
for i in features:
    start = time.time()
    for j in sensors:
        df2[i+'_'+j] = np.where(df2['reporting_area']==j,df2[i],0)
    end = time.time()

In [ ]:
for i in features:
    del df2[i]

In [ ]:
del df2['reporting_area']

In [ ]:
df2.reset_index(drop=True,inplace=True)

In [ ]:
a_sensors = df2.columns.tolist()

In [ ]:
del a_sensors[0]

In [ ]:
d = {k:'sum' for k in a_sensors}

In [ ]:
df_final = df2.groupby('timestamp',as_index=False).agg(d)

In [ ]:
for col in a_sensors:    
    df_final[col] = np.where(df_final[col]==-1.0,np.NaN,df_final[col])

In [ ]:
df_final.index = df_final['timestamp']

In [ ]:
del df_final['timestamp']

In [ ]:
train = df_final.loc['2017-01-01 00:00:00':'2017-12-31 23:00:00',:]
test = df_final.loc['2018-01-01 00:00:00':'2018-02-28 23:00:00',:]

In [ ]:
train.isnull().sum().sum()

In [ ]:
for col in train.columns:
    train['temp'] = pd.rolling_mean(arg=train[col],center=True,min_periods=1,window=7)
    train[col] = np.where(train[col].isnull(),train['temp'],train[col])
    del train['temp']

In [ ]:
train.isnull().sum().sum()

In [ ]:
df_final = pd.concat([train,test],axis=0)

In [ ]:
df_final.shape

In [ ]:
df_final.reset_index(inplace=True)

In [ ]:
df_final.to_csv('processed_los_angeles_weather_14_months.csv',index=False)